# EQUALWEIGHT PART 



In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
#type(stocks) -> dataframe check
#stocks -> you can check stocks that way

the most api's work is they only expose certain data through each endpoint, which makes things faster if you only need to retrieve certain data


In [ ]:
from secrets import IEX_CLOUD_API_TOKEN #if this code fails, just restart jupyterkernel
#secrets.py is made for tokens by Python
#IEX Cloud provides all accounts a free, unlimited use sandbox for testing. 
# Every account will be assigned two test tokens available via the Console. 
# All sandbox endpoints function the same as production, so you will only need to change the base url and token

In [ ]:
""" 
we need market capitalizm for each stock
price of each stock
we use sandbox free api. 
sandbox api is free and we can improve
real investment strategies
google -> iex cloud api -> find sandbox base url
"""
symbol = 'AAPL'                         
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' 
data = requests.get(api_url).json()
#type(data) -> response
#print(data.status_code) ->check data status.
print(data)                                 
                                        
                                        


In [ ]:
data['latestPrice']
data['marketCap']

In [ ]:
my_columns = [ 'Ticker' , 'Price' , 'Market Capitalization' , 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

In [ ]:
final_dataframe = final_dataframe.append(
    pd.Series(
        [
            'AAPL', 
            data['latestPrice'], 
            data['marketCap'], 
            'N/A'], 
            index = my_columns), 
            ignore_index = True)
final_dataframe
   
#we have pandas series and dataframe problems in there(index,ignore_index)

### we need to do now is loop to every ticker in our stocks variable and execute an API call for that stock, and then store the results of that API call in our pandas dataframe” with 'for'

this loop  running very slow because of http request

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' 
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data['latestPrice'],
                data['marketCap'],
                'N/A' #for the last row
            ],
            index = my_columns
        ),
        ignore_index=True
    )

final_dataframe

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst),n):
        yield lst[i:i +n]

## we need to transform all of the stocks that's in each of those lists into a string, and that string will be passed into the URL of the HTTP request that we're executing

first, we'll just create an empty list called symbol strings. and its going to be a list of strings, where each string is a comma separated string of all the stocks in this object (symbol_strings)
we need to do is loop through every string in our symbol strings object, and then use that string to execute a batch API request. So we're gonna use a for loop for this. And to keep the nomenclature pretty simple we're going to just say for symbol string in symbol

/stock/market/batch?symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5 -> from iex api cloud documentation


In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100)) #iex cloud api batch limit is 100 ticker
symbol_strings = []
#symbol_groups
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
#print(symbol_strings)    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

final_dataframe

# calculating the number of shares to buy

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

print(final_dataframe)
#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

In [ ]:
#formatting our excel output
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
#creating the formats we'll need for our .xlsx file
#String format for tickers
#\$XX.XX format for stock prices
#\$XX,XXX format for market capitalization
#Integer format for the number of shares to purchase
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
#applying the formats to the columns of our .xlsx file
#example:
"""
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
18, #This tells the method to apply a column width of 18 pixels
string_format #This applies the format 'string_format' to the column
)
"""
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


In [ ]:
#simplify this by putting it in 2 loops:
column_formats = { 
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
#saving our excel
writer.save()